In [1]:
%matplotlib inline 


import numpy as np

import pandas as pd
from lid_driven_cavity_flow_pinn.utils import generate_csv_catalog, read_datafile, get_boundary_samples
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# models
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import torch



In [2]:
# CUDA support
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

cuda


## Generate the dataset

In this randomly generated dataset I decided to have just a tablular type output with 50,000 samples, 60 features (highly dimensional), and 25 being redundent information for the random and boosted trees.

In [3]:
# Generate the dataset with the number of samples and features with some being redundent and others being useful information
# Input, Response = make_classification(n_samples=50000, n_features=35+25, n_informative=35, n_redundant=25, random_state=7406)

catalog = generate_csv_catalog()

catalog

                        filepath   Re  xsize  ysize
1879      ../data/Re300/PUV0.txt  300    151    151
1880    ../data/Re300/PUV100.txt  300    151    151
1881   ../data/Re300/PUV1000.txt  300    151    151
1882  ../data/Re300/PUV10000.txt  300    151    151
1883  ../data/Re300/PUV10100.txt  300    151    151 
 number of files cataloged: 4850



,filepath,Re,xsize,ysize
1879,../data/Re300/PUV0.txt,300,151,151
1880,../data/Re300/PUV100.txt,300,151,151
1881,../data/Re300/PUV1000.txt,300,151,151
1882,../data/Re300/PUV10000.txt,300,151,151
1883,../data/Re300/PUV10100.txt,300,151,151
...,...,...,...,...
1874,../data/Re2000/PUV9500.txt,2000,151,151
1875,../data/Re2000/PUV9600.txt,2000,151,151
1876,../data/Re2000/PUV9700.txt,2000,151,151
1877,../data/Re2000/PUV9800.txt,2000,151,151


In [4]:
# pull out a single file and check it out.
P, U, V, time, Re = read_datafile(catalog.iloc[1800]["filepath"])
print(
    "total data points in a single file",
    P.flatten().shape[0] + U.flatten().shape[0] + V.flatten().shape[0],
)

total data points in a single file 68403


In [5]:
get_boundary_samples(P).shape[0]

600

In [6]:
# computational boundary
x_lower, x_upper, y_lower, y_upper = 0, 1, -1, 0

# make a new dataframe with all the data in it
P_list, U_list, V_list, time_list, Re_list = [], [], [], [], []
P_boundary_list, U_boundary_list, V_boundary_list = [], [], []
catalog_dict = catalog.to_dict("records")
for file in tqdm(catalog_dict):
    P, U, V, time, Re = read_datafile(file["filepath"])

    # stash all of them
    P_list.append(P.flatten())
    U_list.append(U.flatten())
    V_list.append(V.flatten())
    # select just the boundary points
    P_boundary_list.append(get_boundary_samples(P))
    U_boundary_list.append(get_boundary_samples(U))
    V_boundary_list.append(get_boundary_samples(V))
    # don't forget the time and response variable
    time_list.append(time.flatten())
    Re_list.append(Re)  # response variable

P_df = pd.DataFrame(P_list)
U_df = pd.DataFrame(U_list)
V_df = pd.DataFrame(V_list)
P_boundary_df = pd.DataFrame(P_boundary_list)
U_boundary_df = pd.DataFrame(U_boundary_list)
V_boundary_df = pd.DataFrame(V_boundary_list)
# time_df = pd.DataFrame(time_list)
Re_df = pd.DataFrame(Re_list)


# put together the whole array
P_df_al = pd.concat([P_df, pd.DataFrame(time_list), pd.DataFrame(Re_list)], axis=1)
U_df_al = pd.concat([U_df, pd.DataFrame(time_list), pd.DataFrame(Re_list)], axis=1)
V_df_al = pd.concat([V_df, pd.DataFrame(time_list), pd.DataFrame(Re_list)], axis=1)

# put together just the boundary list
P_boundary_df_al = pd.concat(
    [P_boundary_df, pd.DataFrame(time_list), pd.DataFrame(Re_list)], axis=1
)
U_boundary_df_al = pd.concat(
    [U_boundary_df, pd.DataFrame(time_list), pd.DataFrame(Re_list)], axis=1
)
V_boundary_df_al = pd.concat(
    [V_boundary_df, pd.DataFrame(time_list), pd.DataFrame(Re_list)], axis=1
)

  0%|          | 0/4850 [00:00<?, ?it/s]

In [ ]:
# do some relabeling
Re_df = Re_df.rename(columns={0: "Re"})
Re_df.Re.astype("category")

Re_dummy = pd.get_dummies(Re_df.Re)
Re_dummy

In [8]:
V_df_al_arr = P_df_al.iloc[:, :-1].to_numpy()
# norm_pres = V_df_al_arr/np.linalg.norm(V_df_al_arr)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1, figsize=(24, 24))
im = ax.imshow(P_df_al.iloc[:, :-1].to_numpy(), cmap="jet", interpolation="nearest")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im, orientation="vertical", cax=cax)
plt.ylabel("time + Re (300 - 2,000)")
plt.xlabel("flattened P velocity")
plt.savefig(
    "../images/P_flattened_y_time_Re_colorbar.png", bbox_inches="tight", dpi=300
)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1, figsize=(24, 24))
im = ax.imshow(U_df_al.iloc[:, :-1].to_numpy(), cmap="jet", interpolation="nearest")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im, orientation="vertical", cax=cax)
plt.ylabel("time + Re (300 - 2,000)")
plt.savefig(
    "../images/U_flattened_y_time_Re_colorbar.png", bbox_inches="tight", dpi=300
)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1, figsize=(24, 24))
im = ax.imshow(V_df_al.iloc[:, :-1].to_numpy(), cmap="jet", interpolation="nearest")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im, orientation="vertical", cax=cax)
plt.ylabel("time + Re (300 - 2,000)")
plt.xlabel("flattened V velocity")
plt.savefig(
    "../images/V_flattened_y_time_Re_colorbar.png", bbox_inches="tight", dpi=300
)

In [ ]:
plt.imshow(np.array(P_list))

From the output above it's worth noting the dataset is binary! (either 0 or 1)

Now let's split this into a test and train split, and do stratified cross-validation on the training dataset!

# Random Forest Classifier

Alrighty, now that the test and train data is split, let's train some models!

In [9]:
# Setup the data, train first half, test second half
P_bound_data_train = np.split(P_boundary_df_al, 2, axis=0)[0]
P_bound_data_test = np.split(P_boundary_df_al, 2, axis=0)[1]
U_bound_data_train = np.split(U_boundary_df_al, 2, axis=0)[0]
U_bound_data_test = np.split(U_boundary_df_al, 2, axis=0)[1]
V_bound_data_train = np.split(V_boundary_df_al, 2, axis=0)[0]
V_bound_data_test = np.split(V_boundary_df_al, 2, axis=0)[1]

# all
print("all, shape", P_df_al.shape)

Input_all_train_p = np.split(P_df_al, 2, axis=0)[0]
Input_all_test_p = np.split(P_df_al, 2, axis=0)[1]
Input_all_train_u = np.split(U_df_al, 2, axis=0)[0]
Input_all_test_u = np.split(U_df_al, 2, axis=0)[1]
Input_all_train_v = np.split(V_df_al, 2, axis=0)[0]
Input_all_test_v = np.split(V_df_al, 2, axis=0)[1]

Input_train_all = pd.concat(
    [Input_all_train_p, Input_all_train_u, Input_all_train_v], axis=1
)
Input_test_all = pd.concat(
    [Input_all_test_p, Input_all_test_u, Input_all_test_v], axis=1
)

# boundary
Input_boundary_train_p = np.split(P_boundary_df_al, 2, axis=0)[0]
Input_boundary_test_p = np.split(P_boundary_df_al, 2, axis=0)[1]
Input_boundary_train_u = np.split(U_boundary_df_al, 2, axis=0)[0]
Input_boundary_test_u = np.split(U_boundary_df_al, 2, axis=0)[1]
Input_boundary_train_v = np.split(V_boundary_df_al, 2, axis=0)[0]
Input_boundary_test_v = np.split(V_boundary_df_al, 2, axis=0)[1]

Input_boundary_train = pd.concat(
    [Input_boundary_train_p, Input_boundary_train_u, Input_boundary_train_v], axis=1
)
Input_boundary_test = pd.concat(
    [Input_boundary_test_p, Input_boundary_test_u, Input_boundary_test_v], axis=1
)


# boundary with 10% randomly selected
rand_select = np.random.randint(P_bound_data_train.shape[1], size=60)
Input_boundary_10_random_train_p = P_bound_data_train[
    np.intersect1d(P_bound_data_train.columns, rand_select)
]
Input_boundary_10_random_test_p = P_bound_data_test[
    np.intersect1d(P_bound_data_train.columns, rand_select)
]
Input_boundary_10_random_train_u = U_bound_data_train[
    np.intersect1d(U_bound_data_train.columns, rand_select)
]
Input_boundary_10_random_test_u = U_bound_data_test[
    np.intersect1d(U_bound_data_train.columns, rand_select)
]
Input_boundary_10_random_train_v = V_bound_data_train[
    np.intersect1d(V_bound_data_train.columns, rand_select)
]
Input_boundary_10_random_test_v = V_bound_data_test[
    np.intersect1d(V_bound_data_train.columns, rand_select)
]

Input_boundary_10_random_train = pd.concat(
    [
        Input_boundary_10_random_train_p,
        Input_boundary_10_random_train_u,
        Input_boundary_10_random_train_v,
    ],
    axis=1,
)
Input_boundary_10_random_test = pd.concat(
    [
        Input_boundary_10_random_test_p,
        Input_boundary_10_random_test_u,
        Input_boundary_10_random_test_v,
    ],
    axis=1,
)


# print(Input_all_train.dtype)
Re_dummy_train = np.split(Re_dummy, 2, axis=0)[0]
Re_dummy_test = np.split(Re_dummy, 2, axis=0)[1]
Re_train = np.split(Re_df, 2, axis=0)[0]
Re_test = np.split(Re_df, 2, axis=0)[1]
print(
    Input_train_all.shape,
    Input_boundary_train.shape,
    Input_boundary_10_random_train.shape,
    Re_dummy_train.shape,
)
print(type(Input_train_all), type(Re_dummy_train))

all, shape (4850, 45603)
(2425, 136809) (2425, 70206) (2425, 183) (2425, 18)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [33]:
Input_train_all.dtypes

0        float64
1        float64
2        float64
3        float64
4        float64
          ...   
22797    float64
22798    float64
22799    float64
22800    float64
0          int64
Length: 68406, dtype: object

In [ ]:
RandomForestModel_all = RandomForestClassifier(
    n_jobs=-1, n_estimators=10000, bootstrap=True, random_state=7406
).fit(Input_train_all.to_numpy()[:, :-1], Re_dummy_train)
print(RandomForestModel_all)
RandomForestModel_boundary = RandomForestClassifier(
    bootstrap=True, n_estimators=10000, n_jobs=-1, random_state=7406
).fit(Input_boundary_train.to_numpy()[:, :-1], Re_dummy_train)
RandomForestModel_boundary_10_random_select = RandomForestClassifier(
    n_estimators=10000, bootstrap=True, n_jobs=-1, random_state=7406
).fit(Input_boundary_10_random_train.to_numpy()[:, :-1], Re_dummy_train)

In [ ]:
# assess the model
# Create confusion matrix
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

# print(confusion_matrix(Re_dummy_test, RandomForestModel_all.predict(Input_test_all.to_numpy()[:, :-1])))
# Display accuracy score
print(
    accuracy_score(
        Re_dummy_test, RandomForestModel_all.predict(Input_test_all.to_numpy()[:, :-1])
    )
    * 100
)
print(
    accuracy_score(
        Re_dummy_test,
        RandomForestModel_boundary.predict(Input_boundary_test.to_numpy()[:, :-1]),
    )
    * 100
)
print(
    accuracy_score(
        Re_dummy_test,
        RandomForestModel_boundary_10_random_select.predict(
            Input_boundary_10_random_test.to_numpy()[:, :-1]
        ),
    )
    * 100
)
# Display F1 score
print(
    "f1 score:",
    f1_score(
        Re_dummy_test,
        RandomForestModel_boundary.predict(
            Input_boundary_10_random_test.to_numpy()[:, :-1]
        ),
        average="weighted",
    ),
)

In [ ]:
GradBoostingClassModel_all = GradientBoostingClassifier(
    n_estimators=100, random_state=7406
).fit(Input_train_all.to_numpy()[:, :-1], Re_train.to_numpy())

GradBoostingClassModel_boundary = GradientBoostingClassifier(
    n_estimators=100, random_state=7406
).fit(Input_boundary_train.to_numpy()[:, :-1], Re_train.to_numpy())
GradBoostingClassModel_boundary_10_random_select = GradientBoostingClassifier(
    n_estimators=100, random_state=7406
).fit(Input_boundary_10_random_train.to_numpy()[:, :-1], Re_train.to_numpy())

/home/bartelsaa/miniconda3/envs/lid_driven_cavity_flow_pin/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bartelsaa/miniconda3/envs/lid_driven_cavity_flow_pin/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bartelsaa/miniconda3/envs/lid_driven_cavity_flow_pin/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [35]:
# assess the model
# Create confusion matrix
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

# print(confusion_matrix(Re_dummy_test, RandomForestModel_all.predict(Input_test_all.to_numpy()[:, :-1])))
# Display accuracy score
print(
    accuracy_score(
        Re_test, GradBoostingClassModel_all.predict(Input_test_all.to_numpy()[:, :-1])
    )
    * 100
)
print(
    accuracy_score(
        Re_test,
        GradBoostingClassModel_boundary.predict(Input_boundary_test.to_numpy()[:, :-1]),
    )
    * 100
)
print(
    accuracy_score(
        Re_test,
        GradBoostingClassModel_boundary_10_random_select.predict(
            Input_boundary_10_random_test.to_numpy()[:, :-1]
        ),
    )
    * 100
)
# Display F1 score
pred_Re_gradBoost = GradBoostingClassModel_boundary_10_random_select.predict(
    Input_boundary_10_random_test.to_numpy()[:, :-1]
)
print(pred_Re_gradBoost.shape)
print(
    "shape:",
    Input_boundary_10_random_test.to_numpy()[:, :-1].shape,
    Input_boundary_10_random_train.to_numpy()[:, :-1].shape,
)
print("f1 score:", f1_score(Re_test, pred_Re_gradBoost, average="weighted"))
# print("f1 score:", f1_score(Re_test, GradBoostingClassModel_boundary.predict(Input_boundary_10_random_test.to_numpy()[:, :-1]), average='weighted'))
# print("f1 score:", f1_score(Re_test, GradBoostingClassModel_boundary_10_random_select.predict(Input_boundary_10_random_test.to_numpy()[:, :-1]), average='weighted'))

NameError: name 'GradBoostingClassModel_all' is not defined